In [ ]:
%matplotlib widget
import logging
logging.basicConfig(level="INFO")

# Creating Protein Meshes in Graphein

Graphein provides functionality to create meshes of protein surfaces. The mesh computation is handled under the hood by PyMol to produce `.obj` files, which are handled by many geometric analysis libraries (such as PyTorch3D). We'll create a few protein meshes, convert them to PyTorch3D objects and visualises them. Again, there is a high-level and low-level API for convenience.

## High-level API
### Config
Again, we use a config object to manage global configurations.

* `pymol_command_line_options`: String of additional pymol command line options when launching. A full list can be found [here](https://www.pymolwiki.org/index.php/Command_Line_Options). The default options, `"-cKq"`, do:
   * `c`: launch in command-line only mode for batch processing
   * `K`: keep alive: when running without a GUI, don't quit after the input
            is exhausted
    * `q`: supress startup message 
    
* `pymol_commands` : List of string pymol commands to execute. You can use these to configure the exact mesh you wish to construct. A reference to these can be found [here](https://pymol.org/pymol-command-ref.html)

In [ ]:
from graphein.protein.config import ProteinMeshConfig
config = ProteinMeshConfig()
config.dict()

### Building a mesh

Let's build a mesh!

In [ ]:
from graphein.protein.meshes import create_mesh

verts, faces, aux = create_mesh(pdb_code="3eiy", config=config)
print(verts, faces, aux)

In order to visualise these tensors containing vertices, faces and aux data, we convert them into a PyTorch3D `Mesh` object

In [ ]:
from graphein.protein.meshes import convert_verts_and_face_to_mesh
m = convert_verts_and_face_to_mesh(verts, faces)

from graphein.protein.visualisation import plot_pointcloud
plot_pointcloud(m, title="my first mesh")


Let's see what happens if we play around with some of the parameters and make a smoother mesh

In [ ]:
pymol_commands = {"pymol_commands": ["hide cartoon",
                                     "set solvent_radius, 4",
                                     "alter all, vdw=4",
                                     "sort",
                                     "set surface_quality, 1",
                                     "show surface"]}

config = ProteinMeshConfig(**pymol_commands)

verts, faces, aux = create_mesh(pdb_code="3eiy", config=config)

m = convert_verts_and_face_to_mesh(verts, faces)

plot_pointcloud(m, title="my second mesh")